## INFOH515 Pyspark code
## Author: Gianluca Bontempi
## Pyspark implementation of the bagging algorithm in the INFOH515 slides "Map-reduce analytics" 

In [2]:
import numpy as np
import pwd
import getpass
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum
from pyspark.mllib.tree import RandomForest, RandomForestModel
from sklearn import linear_model
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier


# create an instance of SparkSession
spark=SparkSession.builder.appName('s.com').getOrCreate()
sc=spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/17 16:37:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/17 16:37:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/17 16:37:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/03/17 16:37:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/03/17 16:37:13 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/03/17 16:37:13 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/03/17 16:37:13 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


In [3]:

np.random.seed(1225)   

n=20 # number of features



Ntr=5000
Nts=1000
N=Ntr+Nts
n=8
X= np.random.normal(loc=0, scale=1, size=N * n).reshape(N, n)
X=np.concatenate((np.ones((N,1)),X), axis=1)
Y=2+X[:,1]-3*X[:,7]+np.random.normal(loc=0, scale=0.1, size=N )
Y=Y.reshape(N, 1)


Xtr=X[:Ntr,]
Ytr=Y[:Ntr,]
Xts=X[Ntr:,]
Yts=Y[Ntr:,]
Ytr.shape=(Ntr,1)
YX=np.hstack((Ytr,Xtr))
vYts=np.var(Yts)
YXrdd=sc.parallelize(YX,10)


In [ ]:

def rddCreateModels(iterator,mod):

    data=np.array(list(iterator))
    X = data[:, 1:]
    Y=data[:, 0]
    m=mod.fit(X,Y)  
    return [m]

def rddUseModel(iterator,Xts):
    rfit=list(iterator)[0]
    #return [pow(Yts-rfit.predict(Xts),2)]
    return [rfit.predict(Xts)]

def rddApplyMean(D,axis=0):
    if (axis==0): # column
        N=D.count()
        return(D.reduce(lambda x,y:x+y)/N)

    if (axis==1): #row
        return(rddArr(D.map(lambda x:mean(x))))
    
nT=1000
mD=15
ncores=3

mregr0= RandomForestRegressor(n_estimators=nT,max_depth=mD,n_jobs=ncores)
M2=YXrdd.mapPartitions(lambda x: rddCreateModels(x,mregr0))

